In [10]:
import os

import geopandas as gpd
import numpy as np
import tobler

from math import floor
from tqdm.auto import tqdm

In [3]:
morphotope_folder = "/data/uscuni-ulce/processed_data/morphotope_clusters/"
destination = "/data/uscuni-ulce/grid_250/"

In [4]:
os.mkdir(destination)
os.mkdir(destination + "regions")

In [5]:
regions = gpd.read_parquet("/data/uscuni-ulce/regions/cadastre_regions_hull.parquet")

In [23]:
def generate_grid(region_id, edge=250):
    morphotopes = gpd.read_parquet(f"{morphotope_folder}{region_id}_clusters.pq")
    xm, ym, xx, yx = morphotopes.total_bounds
    xm = floor(xm / edge) * edge + (edge / 2)
    ym = floor(ym / edge) * edge + (edge / 2)
    xx = floor(xx / edge) * edge + (edge / 2)
    yx = floor(yx / edge) * edge + (edge / 2)
    
    x = np.arange(xm, xx, edge)
    y = np.arange(ym, yx, edge)
    xv, yv = np.meshgrid(x, y)
    
    points = gpd.GeoSeries.from_xy(x=xv.flatten(), y=yv.flatten(), crs=morphotopes.crs).buffer(edge / 2, cap_style=3)
    pts_in, _ = morphotopes.sindex.query(points)
    mask = np.unique(pts_in)
    grid = points[mask].to_frame("geometry")
    interpolated = tobler.area_weighted.area_join(morphotopes, grid, ['final', 'regional', 'final_without_noise'])
    interpolated = interpolated.dropna()
    interpolated.to_parquet(f"{destination}{region_id}_clusters.pq")

In [24]:
for region in tqdm(regions.index[::-1]):
    generate_grid(region)

  0%|          | 0/847 [00:00<?, ?it/s]

/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/tobler/area_weighted/area_join.py:63: UserWarning: Cannot preserve dtype of 'final'. Falling back to `dtype=object`.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/tobler/area_weighted/area_join.py:63: UserWarning: Cannot preserve dtype of 'final_without_noise'. Falling back to `dtype=object`.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/tobler/area_weighted/area_join.py:63: UserWarning: Cannot preserve dtype of 'final'. Falling back to `dtype=object`.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/tobler/area_weighted/area_join.py:63: UserWarning: Cannot preserve dtype of 'final_without_noise'. Falling back to `dtype=object`.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/tobler/area_weighted/area_join.py:63